In [48]:
#libraries used
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
from glob import glob

In [49]:
#getting multichannel masks path 
masks_path = sorted(glob("validation_data/rgb_masks/*.png"))
# print(len(masks_path))
masks_path[0].split('/')[-1]

In [50]:
#Dictionaries to store total tumrous and non tumrous nuclei in images
total_tumrous_nuclei = {}
total_nontumrous_nuclei = {}

In [51]:
#calculating total tumrous nuclei of all images and storing the value in dict along with their names
for i, image in enumerate(masks_path):
    img1 = cv2.imread(image)
    img = img1[:, :, 0]
    ret1, thresh = cv2.threshold(img, 0, 215, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernal = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernal, iterations = 4)
    sure_bg = cv2.dilate(opening, kernal, iterations= 5)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2,3)
    ret2, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(),215,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 10
    markers[unknown==215] = 0
    markers = cv2.watershed(img1, markers)
    img1[markers == -1] = [0, 255, 255]
    img2 = color.label2rgb(markers, bg_label=0)
    
    regions = measure.regionprops(markers, intensity_image = img)
    
    nuclei_number = 1
    for region_props in regions:
        nuclei_number += 1
    
    total_tumrous_nuclei[f"{masks_path[i].split('/')[-1]}"] = nuclei_number

In [52]:
#calculating total non tumrous nuclei of all images and storing the value in dict along with their names
for i, image in enumerate(masks_path):
    img1 = cv2.imread(image)
    img = img1[:, :, 1]
    ret1, thresh = cv2.threshold(img, 0,110, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernal = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernal, iterations = 2)
    sure_bg = cv2.dilate(opening, kernal, iterations= 5)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2,5)
    ret2, sure_fg = cv2.threshold(dist_transform, 0.2*dist_transform.max(),215,0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg,sure_fg)
    ret3, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 10
    markers[unknown==110] = 0
    markers = cv2.watershed(img1, markers)
    img1[markers == -1] = [0, 255, 255]
    img2 = color.label2rgb(markers, bg_label=0)
    
    regions = measure.regionprops(markers, intensity_image = img)
    
    non_tumor_nuclei_number = 1
    for region_props in regions:
        non_tumor_nuclei_number += 1
    
    total_nontumrous_nuclei[f"{masks_path[i].split('/')[-1]}"] = non_tumor_nuclei_number

In [53]:
total_tumrous_nuclei

{'val_p001.png': 402,
 'val_p002.png': 170,
 'val_p003.png': 234,
 'val_p004.png': 271,
 'val_p005.png': 139,
 'val_p006.png': 17,
 'val_p007.png': 85,
 'val_p008.png': 63,
 'val_p009.png': 308,
 'val_p010.png': 116}

In [54]:
total_nontumrous_nuclei

{'val_p001.png': 345,
 'val_p002.png': 226,
 'val_p003.png': 414,
 'val_p004.png': 604,
 'val_p005.png': 118,
 'val_p006.png': 88,
 'val_p007.png': 382,
 'val_p008.png': 124,
 'val_p009.png': 86,
 'val_p010.png': 50}

In [79]:
#calculating tumour cellularity
import pandas as pd
arr = []
for i in range(len(total_tumrous_nuclei)):
    name = masks_path[i].split('/')[-1]
    x = total_tumrous_nuclei[name]
    y = total_nontumrous_nuclei[name]
    
    TC = int((x / (x+y)) * 100)
    t = (name, TC)
    arr.append(t)

In [80]:
#converting the array to dataframe
df= pd.DataFrame(arr, columns=["Name","TC"])

In [81]:
#saving df to csv file
df.to_csv("Tumor_cellularity.csv", index=False)